<a href="https://colab.research.google.com/github/ebaranas/colab/blob/master/MNIST_validation_data_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

class Model(object):
    def __init__(self, model_name, params):
        self.NUM_CLASSES = params['NUM_CLASSES']
        SUPPORTED_MODELS= {"FullyConnected": (self.FullyConnected, [2]),
                    "LowResFrameClassifier": (self.LowResFrameClassifier, [4]),
                    "SimpleFullyConnected":(self.SimpleFullyConnected, [2])
        }
        self.model_function, self.required_dims = SUPPORTED_MODELS.get(model_name, 'KeyError')
        
    def get_model_function(self):
        return self.model_function
    
    def get_required_dims(self):
        # For now first element of list is default.
        return self.required_dims
    
    def train(self, next_batch):
        logit, label = self.model_function(next_batch[0]), next_batch[1]
        loss = self.get_loss(logit, label)
        optimizer = tf.train.AdamOptimizer().minimize(loss)
        accuracy = self.get_accuracy(logit, label)
        return loss, optimizer, accuracy
    
    def get_loss(self, logit, label):
        softmax = tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=logit)
        return tf.reduce_sum(softmax)

    def get_accuracy(self, logit, label):
        prediction = tf.argmax(logit, 1)
        equality = tf.equal(prediction, tf.argmax(label, 1))
        return tf.reduce_mean(tf.cast(equality, tf.float32))
        
    def FullyConnected(self, feature):
        #bn = tf.layers.batch_normalization(feature)
        fc1 = tf.layers.dense(feature, 50)
        fc2 = tf.layers.dense(fc1, 50)
        fc2 = tf.layers.dropout(fc2)
        flat = tf.layers.flatten(fc2) # added flatten layer 08/10/2018 to correct shape
        fc3 = tf.layers.dense(flat, self.NUM_CLASSES)
        return fc3
        
    def SimpleFullyConnected(self, feature):
        #bn = tf.layers.batch_normalization(feature)
        fc1 = tf.layers.dense(feature, 30)
        final = tf.layers.dense(fc1, self.NUM_CLASSES)
        return final
    
    def LowResFrameClassifier(self, feature):
        #bn = tf.layers.batch_normalization(feature)
        conv1 = tf.layers.conv2d(feature, 32, (3, 3), activation="relu")
        conv2 = tf.layers.conv2d(conv1, 32, (3, 3), activation="relu")
        maxpool2d_a = tf.layers.max_pooling2d(conv2, pool_size=(2, 2), strides=(1, 1))
        dropout_a = tf.layers.dropout(maxpool2d_a, rate=0.25)
        
        conv3 = tf.layers.conv2d(dropout_a, 64, (3, 3), activation="relu")
        conv4 = tf.layers.conv2d(conv3, 64, (3, 3), activation="relu")
        maxpool2d_b = tf.layers.max_pooling2d(conv3, pool_size=(2, 2), strides=(1, 1))
        dropout_b = tf.layers.dropout(maxpool2d_b, rate=0.25)
        
        flat = tf.layers.flatten(dropout_b)
        dense = tf.layers.dense(flat, 256, activation="relu")
        dropout_c = tf.layers.dropout(dense, rate=0.25)
        
        final = tf.layers.dense(dropout_c, self.NUM_CLASSES, activation="softmax")
        return final

In [4]:
!pip install python-mnist
from mnist import MNIST

  Running setup.py bdist_wheel for python-mnist ... - done
  Stored in directory: /root/.cache/pip/wheels/28/29/36/408f83545a511c43d03ef997a1dc99b49ccd5f9f306ed92468
Successfully built python-mnist


In [0]:
from mnist import MNIST
import os
import numpy as np

def read_raw_mnist(data_path):
    mndata = MNIST(data_path)
    images, labels = mndata.load_training()
    return np.asarray(images), np.asarray(labels)

def read_raw_bfimage(data_path):
    images = []
    labels = []
    for label_index, label in sorted(enumerate(os.listdir(data_path))):
        filename = data_path + label
        label = label.replace(".npy", "")
        for image in np.load(filename): # will load one npy file which may contain many examples
            images.append(image)
            labels.append(label_index)
    return np.asarray(images), np.asarray(labels)

SUPPORTED_DATA = {"mnist": read_raw_mnist, "bfimage": read_raw_bfimage}

def read_raw(data_name, data_path):
    read = (SUPPORTED_DATA.get(data_name, 'KeyError'))
    return read(data_path)

In [0]:
import os
import time
import numpy as np
import tensorflow as tf
import toml
from tensorflow.python.client import timeline
from multiprocessing import Pool

params = {'DATA_NAME':'mnist', 'DATA_PATH':'.', 'DTYPE':'np.float32', 'HEIGHT':28, 
          'WIDTH':28, 'CHANNELS':1, 'NUM_CLASSES':10, 'BATCH_SIZE':48, 'TRAIN_RATIO':0.8, 'NUM_CORES':4}

class Benchmark(object):
    def __init__(self, model_name):
        print("Training '{model}' on '{data}'".format(model=model_name,
            data=params['DATA_NAME']))
        raw_features, raw_labels = read_raw(params['DATA_NAME'], params['DATA_PATH'])
        
        params.update({'NUM_EXAMPLES': len(raw_features)})
        params.update({'TRAIN_SIZE': int(params['TRAIN_RATIO']*params['NUM_EXAMPLES'])})
        params.update({'VALID_SIZE': int(params['NUM_EXAMPLES'] - params['TRAIN_SIZE'])})
        self.params = params
        self.model = Model(model_name, params)
        
        reshape = self.should_reshape(rank=len(raw_features.shape))
        dtype = eval(params['DTYPE'])
        features, labels = self.preprocess(raw_features, raw_labels, dtype, reshape)
        
        self.full_dataset = tf.data.Dataset.from_tensor_slices((features, labels)).shuffle(self.params['NUM_EXAMPLES'])
     
        print("PARAMS: reshape=", reshape, " from ", raw_features.shape, " dtype=", dtype, " batch size=", 
        params['BATCH_SIZE'], " train size=", params['TRAIN_SIZE'])

    
    def preprocess(self, features, labels, dtype, reshape):
        assert len(features) == len(labels)
        if not isinstance(features, dtype):
            features = dtype(features)
        if reshape is not False:
            features = features.reshape(reshape)
        labels = np.array(Pool(self.params['NUM_CORES']).map(self.encode_to_one_hot, labels))
        return np.asarray(features), np.asarray(labels)
    
    
    def get_dataset(self, mode):
        dataset, buff, batch_size = self.split_dataset(mode)
        dataset = dataset.cache().shuffle(buff).repeat().batch(batch_size).prefetch(1)
        return

      
    def split_dataset(self, mode):
        if mode == 'training':
            return ()
        elif mode == 'validation':
            return (self.full_dataset.skip(self.params['TRAIN_SIZE']), self.params['VALID_SIZE'],
                    self.params['VALID_SIZE'])
        else:
            raise ArgumentError
    
    def encode_to_one_hot(self, label):
        one_hot_encoding = np.zeros(self.params['NUM_CLASSES'])
        one_hot_encoding[label] = 1
        return one_hot_encoding    
    
    def run(self, iterations, profile):
        train_dataset = self.full_dataset.take(self.params['TRAIN_SIZE']).cache().shuffle(
            self.params['TRAIN_SIZE']).repeat().batch(self.params['BATCH_SIZE']).prefetch(1)
        
        valid_dataset = self.full_dataset.skip(self.params['TRAIN_SIZE']).cache().shuffle(
            self.params['VALID_SIZE']).repeat().batch(self.params['VALID_SIZE']).prefetch(1)
        
        # create general iterator
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_batch = iterator.get_next()
        
        # make datasets that we can initialize separately, but using the same structure via the common iterator
        training_init_op = iterator.make_initializer(train_dataset)
        validation_init_op = iterator.make_initializer(valid_dataset)      
        
        model_function = self.model.get_model_function()
        logit, label = model_function(next_batch[0]), next_batch[1]
        loss = self.model.get_loss(logit, label)
        optimizer = tf.train.AdamOptimizer().minimize(loss)
        accuracy = self.model.get_accuracy(logit, label)
        
        if profile is not False:
            # graph_writer = tf.summary.FileWriter("pipeline", sess.graph)
            run_metadata = tf.RunMetadata()
            options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        else:
            run_metadata = None
            options = None
        
        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run([tf.global_variables_initializer()], options=options, run_metadata=run_metadata)
            avg_acc = 0
            print("START BENCHMARK")
            start = time.time()
            for i in range(iterations):
                sess.run(training_init_op)
                loss_, _, acc = sess.run([loss, optimizer, accuracy], options=options, run_metadata=run_metadata)
                sess.run(validation_init_op)
                val_acc = sess.run(accuracy, options=options, run_metadata=run_metadata)
#                 avg_acc += acc
                if i % int(iterations/10) == 0:
                    print("acc, val_acc", acc*100, val_acc*100)
#                     print("Epoch: {}, loss: {:.3f}, {} accuracy: {:.2f}%".format(i, loss_, acc * 100))
#                     print("validation accuracy: {:.2f}%".format(val_acc * 100))
            
            print("Average {} accuracy over {} iterations is {:.2f}%".format(iterations, (avg_acc / iterations) * 100))
            end = time.time()
            time_per_run = end - start
            avg_acc = avg_acc/iterations
            
            print("END BENCHMARK. TIME: ", time_per_run)
        
            self.generate_trace(run_metadata, profile)
            return time_per_run, avg_acc
    
    def generate_trace(self, run_metadata, profile):
        if profile is False:
            return
        else:
            # Create the Timeline object, and write it to a json file
            fetched_timeline = timeline.Timeline(run_metadata.step_stats)
            chrome_trace = fetched_timeline.generate_chrome_trace_format()
            with open(profile, 'w') as f:
                f.write(chrome_trace)
            return

    def should_reshape(self, rank):
        if rank not in self.model.get_required_dims():
            if rank == 4: # e.g. bfimage, FullyConnected
                return (-1, self.params['HEIGHT']*self.params['WIDTH']*self.params['CHANNELS'])
            elif rank == 2: # e.g. mnist, LowResFrameClassifier
                return (-1, self.params['HEIGHT'], self.params['WIDTH'], self.params['CHANNELS'])
            else:
                ValueError("Invalid data shape")
        else:
            return False

def load_params(config_path, param={}):
    '''
    Load parameters from file.
    '''
    params = {}
    if not os.path.isfile(config_path):
        raise KeyError
    else:
        with open(config_path, 'r', encoding='utf-8') as f:
            params.update(toml.load(f))
        return params

In [0]:
import tensorflow as tf
import numpy as np
import os
import sys
from mnist import MNIST
import math

def main():
    # GPU utilization
    NUM_RUNS = 1
    PATH_TO_CHROME_TRACES = "/project/datasets-API/benchmark-package/chrome-traces/test.json"
    
    benchmark = Benchmark('FullyConnected')
    tot_acc, tot_time = 0, 0
    for i in range(NUM_RUNS):
        tot_time += benchmark.run(1000, profile=False)[0]
        # tot_acc += benchmark.run('validation', 1000, profile=False)[1]
    print("RESULTS: Train time=", tot_time/NUM_RUNS, ", Validation accuracy=", tot_acc/NUM_RUNS*100)

if __name__ == "__main__":
    main()

Training 'FullyConnected' on 'mnist'
PARAMS: reshape= False  from  (60000, 784)  dtype= <class 'numpy.float32'>  batch size= 48  train size= 48000
START BENCHMARK
acc, val_acc 0.125 0.068833336
acc, val_acc 0.7291667 0.8269167
acc, val_acc 0.9166667 0.8444167
acc, val_acc 0.7916667 0.8685833
acc, val_acc 0.9166667 0.8699167
acc, val_acc 0.8958333 0.87991667
acc, val_acc 0.875 0.87175
acc, val_acc 0.8541667 0.837
acc, val_acc 0.8333333 0.86466664
